### TF Seq2seq
1) For seq2seq to work we need eos and pad tokens. We use pad as <UNK> since there are no major repurcussions, and eos is stored in vocab already. Unless, <UNK> has a particular embedding. 
2) 

In [1]:
import os
os.chdir('./sequence_tagging/model/')
import tensorflow as tf
import pickle
import numpy as np


/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def get_word_embeddings(filename):
    """
    Args:
        filename: path to the npz file

    Returns:
        matrix of embeddings (np array)

    """
    try:
        with np.load(filename) as data:
            return data["embeddings"]

    except IOError:
        raise MyIOError(filename)
        
def dataset_load(domain_tr_data_path, vocab_path):
    with open(domain_tr_data_path,'r') as p1:
        domain_tr_data = pickle.load(p1)
    with open(vocab_path,'r') as p1:
        vocab = pickle.load(p1)
        
    domain_tr_data = map(lambda x: x[0],domain_tr_data)
    idd_domain_tr_data = map(lambda x: [vocab[word] for word in x], domain_tr_data)
    return idd_domain_tr_data, vocab

In [3]:
domain_name = 'laptop'
domain_tr_data_path = '../data/Final_joint_data_absa//Domains/Laptop/Normal__normal_training_list.pickle'
embeddings_path = '../data/Embeddings/Pruned/np_glove_200d_trimmed.npz'
embeddings_name = 'glove200d'
vocab_path = '../data/vocab_to_id.pkl'


In [4]:
with open(vocab_path,'r') as p1:
        vocab = pickle.load(p1)
word_embeddings_np = get_word_embeddings(embeddings_path)

In [5]:
pad_token = '<START>' #This is due to a mistake on my part. Didn't store PAD in vocab, stored unkown-> might lead to ambiguity. So assume start == pad for now. Will edit vocab later
eos_token = '<END>'
PAD = vocab[pad_token]
EOS = vocab[eos_token]

In [6]:
vocab_size = len(vocab)
input_embedding_size = 200
encoder_hidden_units = 50 #100
decoder_hidden_units = encoder_hidden_units*2

In [7]:
encoder_inputs = tf.placeholder(shape=(None, None), dtype=tf.int32, name='encoder_inputs')
encoder_inputs_length = tf.placeholder(shape=(None,), dtype=tf.int32, name='encoder_inputs_length')
decoder_targets = tf.placeholder(shape=(None, None), dtype=tf.int32, name='decoder_targets')

In [8]:
embeddings = tf.Variable(word_embeddings_np, name="word_embeds",dtype=tf.float32, trainable=False)

In [9]:
encoder_inputs_embedded = tf.nn.embedding_lookup(embeddings, encoder_inputs)

In [10]:
from tensorflow.contrib.rnn import LSTMCell, LSTMStateTuple

In [11]:
encoder_cell = LSTMCell(encoder_hidden_units)

In [12]:
((encoder_fw_outputs,
  encoder_bw_outputs),
 (encoder_fw_final_state,
  encoder_bw_final_state)) = (
    tf.nn.bidirectional_dynamic_rnn(cell_fw=encoder_cell,
                                    cell_bw=encoder_cell,
                                    inputs=encoder_inputs_embedded,
                                    sequence_length=encoder_inputs_length,
                                    dtype=tf.float32, time_major=True)
    )

In [13]:
encoder_fw_outputs

<tf.Tensor 'bidirectional_rnn/fw/fw/TensorArrayStack/TensorArrayGatherV3:0' shape=(?, ?, 50) dtype=float32>

In [14]:
encoder_bw_outputs

<tf.Tensor 'ReverseSequence:0' shape=(?, ?, 50) dtype=float32>

In [15]:
encoder_fw_final_state, encoder_bw_final_state

(LSTMStateTuple(c=<tf.Tensor 'bidirectional_rnn/fw/fw/while/Exit_2:0' shape=(?, 50) dtype=float32>, h=<tf.Tensor 'bidirectional_rnn/fw/fw/while/Exit_3:0' shape=(?, 50) dtype=float32>),
 LSTMStateTuple(c=<tf.Tensor 'bidirectional_rnn/bw/bw/while/Exit_2:0' shape=(?, 50) dtype=float32>, h=<tf.Tensor 'bidirectional_rnn/bw/bw/while/Exit_3:0' shape=(?, 50) dtype=float32>))

In [16]:
encoder_outputs = tf.concat((encoder_fw_outputs, encoder_bw_outputs),2)

encoder_final_state_c = tf.concat(
    (encoder_fw_final_state.c, encoder_bw_final_state.c), 1)

encoder_final_state_h = tf.concat(
    (encoder_fw_final_state.h, encoder_bw_final_state.h), 1)

encoder_final_state = LSTMStateTuple(
    c=encoder_final_state_c,
    h=encoder_final_state_h
) #this is useful later

encoder_final_state_c

<tf.Tensor 'concat_1:0' shape=(?, 100) dtype=float32>

In [17]:
encoder_concat_everything = tf.concat([encoder_final_state_c,encoder_final_state_h], 1)

In [18]:
encoder_concat_everything

<tf.Tensor 'concat_3:0' shape=(?, 200) dtype=float32>

In [19]:
decoder_cell = LSTMCell(decoder_hidden_units)
encoder_max_time, batch_size = tf.unstack(tf.shape(encoder_inputs))

In [20]:
decoder_lengths = encoder_inputs_length + 4 #3 additional 

In [21]:
W = tf.Variable(tf.random_uniform([decoder_hidden_units, vocab_size], -1, 1), dtype=tf.float32)
b = tf.Variable(tf.zeros([vocab_size]), dtype=tf.float32)

In [22]:
eos_time_slice = EOS*tf.ones([batch_size], dtype=tf.int32, name='EOS')
pad_time_slice = PAD*tf.ones([batch_size], dtype=tf.int32, name='PAD')

eos_step_embedded = tf.nn.embedding_lookup(embeddings, eos_time_slice)
pad_step_embedded = tf.nn.embedding_lookup(embeddings, pad_time_slice)

In [23]:
def loop_fn_initial():
    initial_elements_finished = (0 >= decoder_lengths)  # all False at the initial step
    initial_input = eos_step_embedded
    initial_cell_state = encoder_final_state
    initial_cell_output = None
    initial_loop_state = None  # we don't need to pass any additional information
    return (initial_elements_finished,
            initial_input,
            initial_cell_state,
            initial_cell_output,
            initial_loop_state)

def loop_fn_transition(time, previous_output, previous_state, previous_loop_state):

    def get_next_input():
        output_logits = tf.add(tf.matmul(previous_output, W), b)
        prediction = tf.argmax(output_logits, axis=1)
        next_input = tf.nn.embedding_lookup(embeddings, prediction)
        return next_input
    
    elements_finished = (time >= decoder_lengths) # this operation produces boolean tensor of [batch_size]
                                                  # defining if corresponding sequence has ended

    finished = tf.reduce_all(elements_finished) # -> boolean scalar
    input = tf.cond(finished, lambda: pad_step_embedded, get_next_input)
    state = previous_state
    output = previous_output
    loop_state = None

    return (elements_finished, 
            input,
            state,
            output,
            loop_state)

In [24]:
def loop_fn(time, previous_output, previous_state, previous_loop_state):
    if previous_state is None:    # time == 0
        assert previous_output is None and previous_state is None
        return loop_fn_initial()
    else:
        return loop_fn_transition(time, previous_output, previous_state, previous_loop_state)

decoder_outputs_ta, decoder_final_state, _ = tf.nn.raw_rnn(decoder_cell, loop_fn)
decoder_outputs = decoder_outputs_ta.stack()

In [25]:
decoder_outputs

<tf.Tensor 'TensorArrayStack/TensorArrayGatherV3:0' shape=(?, ?, 100) dtype=float32>

In [26]:

decoder_max_steps, decoder_batch_size, decoder_dim = tf.unstack(tf.shape(decoder_outputs))
decoder_outputs_flat = tf.reshape(decoder_outputs, (-1, decoder_dim))
decoder_logits_flat = tf.add(tf.matmul(decoder_outputs_flat, W), b)
decoder_logits = tf.reshape(decoder_logits_flat, (decoder_max_steps, decoder_batch_size, vocab_size))

In [27]:
decoder_prediction = tf.argmax(decoder_logits, 2)

In [28]:

stepwise_cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
    labels=tf.one_hot(decoder_targets, depth=vocab_size, dtype=tf.float32),
    logits=decoder_logits,
)

loss = tf.reduce_mean(stepwise_cross_entropy)


In [29]:
batch_size = 25
num_epochs = 10

def dataset_load(domain_tr_data_path, vocab_path):
    with open(domain_tr_data_path,'r') as p1:
        domain_tr_data = pickle.load(p1)
    with open(vocab_path,'r') as p1:
        vocab = pickle.load(p1)
        
    domain_tr_data = map(lambda x: x[0],domain_tr_data)
    idd_domain_tr_data = map(lambda x: [vocab[word] for word in x], domain_tr_data)
    return idd_domain_tr_data

def get_holdout_data(idd_domain_tr_data, num=10):
    '''Simple function to check if encoder is functioning properly'''
    return idd_domain_tr_data[:num]

def gen_batch(idd_data, batch_size):
    np.random.shuffle(idd_data)
    rem = len(idd_data)%batch_size
    num_batches = (len(idd_data)/batch_size) 
    if rem>0:
        num_batches = num_batches + 1
    
    
    for i in range(num_batches):
        if(i == num_batches -1 and (not rem==0)):
            yield(idd_data[i*batch_size:])
        else:
            yield(idd_data[i*batch_size:(i+1)*batch_size])
            
def batch_modify(inputs, max_sequence_length=None):
    """
    Args:
        inputs:
            list of sentences (integer lists)
        max_sequence_length:
            integer specifying how large should `max_time` dimension be.
            If None, maximum sequence length would be used
    
    Outputs:
        inputs_time_major:
            input sentences transformed into time-major matrix 
            (shape [max_time, batch_size]) padded with 0s
        sequence_lengths:
            batch-sized list of integers specifying amount of active 
            time steps in each input sequence
    """
    
    sequence_lengths = [len(seq) for seq in inputs]
    batch_size = len(inputs)
    
    if max_sequence_length is None:
        max_sequence_length = max(sequence_lengths)
    
    inputs_batch_major = np.zeros(shape=[batch_size, max_sequence_length], dtype=np.int32) # == PAD
    
    for i, seq in enumerate(inputs):
        for j, element in enumerate(seq):
            inputs_batch_major[i, j] = element

    # [batch_size, max_time] -> [max_time, batch_size]
    inputs_time_major = inputs_batch_major.swapaxes(0, 1)

    return inputs_time_major, sequence_lengths
        
def next_feed(batch):
    encoder_inputs_, encoder_input_lengths_ = batch_modify(batch)
    decoder_targets_, _ = batch_modify(
        [(sequence) + [EOS] + [PAD] * 3 for sequence in batch] #additional 3 spaces
    )
    return {
        encoder_inputs: encoder_inputs_,
        encoder_inputs_length: encoder_input_lengths_,
        decoder_targets: decoder_targets_,
    }

def feed_enc(enc_batch):
    encoder_inputs_, encoder_input_lengths_ = batch_modify(enc_batch)
    return {encoder_inputs: encoder_inputs_,
            encoder_inputs_length: encoder_input_lengths_}


In [30]:
idd_data = dataset_load(domain_tr_data_path,vocab_path)
holdout_data = get_holdout_data(idd_data,10)

In [31]:
embed_type = "Glove"
model_path = '../results/seq2seq/{}_seq2seqmodel_embeds{}_{}d_{}hiddenunits.ckpt'.format(domain_name,embed_type,input_embedding_size,encoder_hidden_units)

batch_size = 150
lr = None
num_epochs = 1
model_exists_already = False

if (lr is None):
    train_op = tf.train.AdamOptimizer().minimize(loss)
else:
    train_op = tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)


if not os.path.exists(model_path):
    os.makedirs(model_path)



In [32]:
graph = tf.get_default_graph()
config = tf.ConfigProto(device_count={'GPU': 0})
saver = tf.train.Saver()
encoder_saver = tf.train.Saver()

In [33]:

loss_track = []

def train_graph(model_exists_already):
    
    with tf.Session(config=config) as sess:
        
        sess.run(tf.global_variables_initializer())

        print("Initialized session")
        if(model_exists_already):
            print("loading existing model")
            saver.restore(sess, model_path)
        
        for epoch in range(num_epochs):
            print("At epoch: {}".format(epoch))
            iters = 0 
            epoch_loss = 0.
            batch_generator = gen_batch(idd_data, batch_size)
            for batch in batch_generator:
                
                fd = next_feed(batch)
                _, l = sess.run([train_op, loss], fd)
    
                if(iters%33==0):
                    print("At batch: {}".format(iters))
                    print("Batch loss: {}".format(l))
                loss_track.append(l)
                iters+=1
                epoch_loss+=l
            print("Epoch training loss: {}".format(epoch_loss/iters))
            
        f_enc = feed_enc(holdout_data)
        encoder_useful_state = sess.run(encoder_concat_everything, f_enc)
            
        saver.save(sess,model_path)
        print("Saved model at: {}".format(model_path))
        #encoder_useful_state = sess.run(encoder_concat_everything)
    return encoder_useful_state



In [34]:
encoder_out = train_graph(False)

Initialized session
At epoch: 0
0
At batch: 0
Batch loss: 8.94123077393
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
Epoch training loss: 8.67789202645
Saved model at: ../results/seq2seq/laptop_seq2seqmodel_embedsGlove_200d_50hiddenunits.ckpt


In [35]:
encoder_out

array([[-0.03974946,  0.13104852, -0.15653771, ..., -0.06743222,
         0.17112713,  0.6098332 ],
       [-0.12657294,  0.3494651 , -0.02483254, ..., -0.04309835,
         0.16275495,  0.5704211 ],
       [ 0.01887111,  0.35905957, -0.22151423, ..., -0.07891329,
        -0.06936573,  0.44957113],
       ...,
       [-0.20589386, -0.14179042, -0.11227965, ..., -0.2480825 ,
        -0.0198348 ,  0.38863584],
       [-0.00574304,  0.40867656, -0.02486085, ..., -0.07106452,
         0.14196876,  0.6010943 ],
       [-0.12318017,  0.2642312 , -0.45233864, ..., -0.13476329,
         0.07698715,  0.4307763 ]], dtype=float32)

In [36]:
encoder_out[0]

array([-3.97494584e-02,  1.31048515e-01, -1.56537712e-01,  9.37305212e-01,
        1.85016558e-01, -2.25092173e-01, -1.24445307e+00, -8.87932420e-01,
       -2.27239355e-02,  1.82536650e+00, -1.16929698e+00,  6.41500652e-02,
        7.82994688e-01,  7.97010601e-01, -2.96856618e+00,  1.25599658e+00,
        1.26194859e+00, -1.66895700e+00, -3.04197359e+00, -6.65612936e-01,
        6.79446697e-01,  1.16614021e-01, -5.56325763e-02, -5.54149365e-03,
       -8.14260364e-01,  1.63301587e+00,  5.53404152e-01, -3.52173522e-02,
       -6.87773645e-01, -1.42797542e+00,  9.68344152e-01,  3.15772325e-01,
        3.19913998e-02,  1.56362402e+00,  1.11489224e+00,  5.61491609e-01,
        1.49946332e+00,  1.99195459e-01,  8.97922695e-01,  1.07440539e-02,
        1.04080009e+00,  3.93972397e-01,  2.84967244e-01, -2.83503324e-01,
        3.20904088e+00,  1.52513731e+00, -7.99509764e-01, -4.63211238e-02,
       -8.66600126e-02,  8.65750253e-01, -7.38304555e-02,  2.60569870e-01,
       -5.19262910e-01,  

In [32]:
'''Cell to continue training'''
with tf.Session(config=config) as sess:
    #saver = tf.train.import_meta_graph('../results/seq2seq/laptop_seq2seqmodel_embedsGlove_200d_100hiddenunits.meta')
    #saver.restore(sess, tf.train.latest_checkpoint('../results/seq2seq/laptop_seq2seqmodel_embedsGlove_200d_100hiddenunits.data-00000-of-00001'))
    #sess.run()
    saver.restore(sess, model_path)
    batch_generator = gen_batch(idd_data, batch_size)
    for batch in batch_generator:        
        fd = next_feed(batch)
        _, l = sess.run([train_op, loss], fd)
        encoder_useful_state = sess.run(encoder_concat_everything, fd)

"Cell to continue training\nwith tf.Session(config=config) as sess:\n    #saver = tf.train.import_meta_graph('../results/seq2seq/laptop_seq2seqmodel_embedsGlove_200d_100hiddenunits.meta')\n    #saver.restore(sess, tf.train.latest_checkpoint('../results/seq2seq/laptop_seq2seqmodel_embedsGlove_200d_100hiddenunits.data-00000-of-00001'))\n    #sess.run()\n    saver.restore(sess, model_path)\n    batch_generator = gen_batch(idd_data, batch_size)\n    for batch in batch_generator:        \n        fd = next_feed(batch)\n        _, l = sess.run([train_op, loss], fd)\n        encoder_useful_state = sess.run(encoder_concat_everything, fd)\n        \n        "

In [33]:
'''Cell to test in format for experiment'''
def restored_model_enc_out(model_exists_already=True):
    with tf.Session(config=config) as sess:
        
        sess.run(tf.global_variables_initializer())

        print("Initialized session")
        if(model_exists_already):
            print("loading existing model")
            saver.restore(sess, model_path)
        
       
        f_enc = feed_enc(holdout_data)
        encoder_useful_state = sess.run(encoder_concat_everything, f_enc)
            
        #encoder_useful_state = sess.run(encoder_concat_everything)
    return encoder_useful_state

In [34]:
encoder_useful_state = restored_model_enc_out()

Initialized session
loading existing model
INFO:tensorflow:Restoring parameters from ../results/seq2seq/laptop_seq2seqmodel_embedsGlove_200d_50hiddenunits.ckpt


In [35]:
encoder_useful_state

array([[-0.03974946,  0.13104852, -0.15653771, ..., -0.06743222,
         0.17112713,  0.6098332 ],
       [-0.12657294,  0.3494651 , -0.02483254, ..., -0.04309835,
         0.16275495,  0.5704211 ],
       [ 0.01887111,  0.35905957, -0.22151423, ..., -0.07891329,
        -0.06936573,  0.44957113],
       ...,
       [-0.20589386, -0.14179042, -0.11227965, ..., -0.2480825 ,
        -0.0198348 ,  0.38863584],
       [-0.00574304,  0.40867656, -0.02486085, ..., -0.07106452,
         0.14196876,  0.6010943 ],
       [-0.12318017,  0.2642312 , -0.45233864, ..., -0.13476329,
         0.07698715,  0.4307763 ]], dtype=float32)